In [2]:
from Company_Lister import Company_Lister
from Daily_Preprocessor import Daily_Preprocessor
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import util
from Evaluator import Evaluator

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout

import numpy as np
import pandas as pd

X, Y, price = Daily_Preprocessor('GOOGL').produce_ind_and_response(stacked=True)
p_change_sd = Y.std()

def assign_to_bins(val, separators):
    i = 0
    while i < len(separators) and val > separators[i]:
        i += 1
    return i

def assign_to_top_bin(val, separators):
    top_sep = separators[len(separators) - 1]
    if val > top_sep:
        return 1
    return 0

Y_binned = np.vectorize(assign_to_top_bin, excluded=[1])(Y, [1 - p_change_sd, 1, 1 + p_change_sd])
num_good = Y_binned.sum()
bad_indices = np.where(Y_binned == 0)[0]
good_indices = np.where(Y_binned == 1)[0]
keep = np.concatenate((np.random.choice(bad_indices, num_good), good_indices))

Y_one_hot = Y_binned[:, np.newaxis] == np.arange(2)[np.newaxis, :]


In [3]:
### Do min max scaling on input
def three_d_min_max(X, mins=None, maxs=None):
    all_arr = np.concatenate(X, axis=0)
    if mins is None or maxs is None:
        mins = all_arr.min(axis=0)
        maxs = all_arr.max(axis=0)
    all_norm =  (X - mins[np.newaxis, :])/((maxs-mins)[np.newaxis,:])
    return all_norm.reshape(X.shape), mins, maxs

In [7]:
model = eval('Sequential()')
exec("model.add(LSTM(20, input_shape=(train_x.shape[1],train_x.shape[2])))")
exec("model.add(Dropout(0.4))")
exec("model.add(Dense(10))")
exec("model.add(Dense(2, activation='softmax'))")
exec("model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])")
model.summary


NameError: name 'train_x' is not defined

In [102]:
train_x, test_x, train_y, test_y = train_test_split(X, Y_one_hot)
train_x, train_mins, train_maxs = three_d_min_max(train_x)
num_good = train_y[:, 1].sum()
bad_indices = np.where(train_y[:, 1] == 0)[0]
good_indices = np.where(train_y[:, 1] == 1)[0]
keep = np.concatenate((np.random.choice(bad_indices, int(num_good * 1.5)), good_indices))
train_x = train_x[keep]
train_y = train_y[keep]
test_x, _, __= three_d_min_max(test_x, train_mins, train_maxs)

model = Sequential()
model.add(LSTM(20, input_shape=(train_x.shape[1],train_x.shape[2])))
model.add(Dropout(0.4))
model.add(Dense(10))
model.add(Dense(2, activation='softmax'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(train_x, train_y, epochs=26, batch_size=32, validation_data=(test_x, test_y))


Epoch 1/26
3/3 [==============================] - 2s 206ms/step - loss: 0.6951 - accuracy: 0.4713 - val_loss: 0.6960 - val_accuracy: 0.5217
Epoch 2/26
3/3 [==============================] - 0s 22ms/step - loss: 0.6960 - accuracy: 0.5747 - val_loss: 0.6948 - val_accuracy: 0.5217
Epoch 3/26
3/3 [==============================] - 0s 22ms/step - loss: 0.6945 - accuracy: 0.5632 - val_loss: 0.6938 - val_accuracy: 0.5217
Epoch 4/26
3/3 [==============================] - 0s 21ms/step - loss: 0.6818 - accuracy: 0.5862 - val_loss: 0.6929 - val_accuracy: 0.5217
Epoch 5/26
3/3 [==============================] - 0s 28ms/step - loss: 0.6667 - accuracy: 0.6667 - val_loss: 0.6914 - val_accuracy: 0.5652
Epoch 6/26
3/3 [==============================] - 0s 24ms/step - loss: 0.6627 - accuracy: 0.6207 - val_loss: 0.6900 - val_accuracy: 0.5652
Epoch 7/26
3/3 [==============================] - 0s 21ms/step - loss: 0.6700 - accuracy: 0.6207 - val_loss: 0.6881 - val_accuracy: 0.6087
Epoch 8/26
3/3 [==========

In [103]:
pred = model.predict(test_x)
ev = Evaluator('classification')
ev.get_metrics(pred, test_y)

{'Accuracy': 0.605334,
 'Precision_0': 0.55059826,
 'Recall_0': 0.84277534,
 'Precision_1': 0.52573705,
 'Recall_1': 0.8190738}

In [105]:
pred = model.predict(test_x)
p = np.full(pred.shape, 0)
p[np.arange(pred.shape[0]), np.argmax(pred, axis=1)] = 1
pd.DataFrame(pred)


,0,1
0,0.537385,0.462615
1,0.938344,0.061656
2,0.624090,0.375910
3,0.963188,0.036812
4,0.934901,0.065099
5,0.851779,0.148221
6,0.534314,0.465686
7,0.443094,0.556907
8,0.552079,0.447921
9,0.034250,0.965750


In [106]:
pd.DataFrame(test_y)


,0,1
0,False,True
1,True,False
2,True,False
3,False,True
4,True,False
5,True,False
6,False,True
7,True,False
8,False,True
9,True,False


In [4]:
import yfinance as yf
t = yf.Ticker('A')
df = t.history(start='2011-01-01', end='2021-10-31', interval='1d')
df.head()

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2011-01-03,27.169147,27.548312,27.071087,27.378342,4994075,0.0,0.0
2011-01-04,27.450244,27.522155,26.920721,27.123379,5017282,0.0,0.0
2011-01-05,26.973020,27.280273,26.803049,27.064541,4519035,0.0,0.0
2011-01-06,27.044932,27.352185,26.940333,27.116842,4699097,0.0,0.0
2011-01-07,27.142989,27.326033,26.829195,27.208363,3810948,0.0,0.0


In [5]:
df['Dividends'].mean()

0.0018552160674981657

In [6]:
df['Stock Splits'].mean()

0.0005128393250183418